# Main compound analisis after compound dicovery step

Requirements: 
- Exported compounds database in excel format
- Exported Chemspider results database in excel format


In [ ]:
import pandas as pd
import chemspi_web_db as chemsearch
from chemspi_web_db import ChemspiWebDB
from chemspi_local_db import ChemspiLocalDB
import compound_analisis_utils as utils

In [ ]:
# Input
folder = "D:/NAS/TEST/input data/"
compounds_database_file = folder + "Tissue_pos_neg_CSID_RR.xlsx"

web_api_key = "9GS3pzBwGsrdu0agqqP7buFcYwaaX2GH"
chemspider_local_db_file = folder + "Tissue_melanoma_negativo_080520_CS2.xlsx"

export_file = folder + "Tissue_pos_neg_CSID_RR_cachedv2"

In [ ]:
# init resources and databases
compounds_table = pd.read_excel(compounds_database_file)
chempider_local_db = ChemspiLocalDB(chemspider_local_db_file)
chempider_web_db = chemsearch.ChemspiWebDB(web_api_key)

# remove not needed columns
#compounds_table.drop(columns=['Checked'], inplace=True)

print("Sucess reading databases")
display(compounds_table)

## Step 1: Removing duplicated entries
- Sort by Name
- Sort by area
- Sort by retention time
- Select from duplicated names: one with highest area then retention time

In [ ]:

compounds_table = utils.remove_duplicated_entries(compounds_table)
print("Done")
compounds_table.to_excel(export_file + "_step1.xlsx")
display(compounds_table)

# Step 2: Find CSId from chemspi databases
- Try local first
- Try online otherwise

In [ ]:
#compounds_table = pd.read_excel(export_file + "_step1.xlsx")

csid_values = []
iupac_values = []

num_rows = compounds_table.shape[0]
row_count = 0
for row_index, row_data in compounds_table.iterrows() :    
    row_count += 1
    compound_name = str(row_data.Name).lower()
    compound_mass = row_data['Molecular Weight']
    print('({:d}/{:d}) searching compound info '.format(row_count, num_rows) + compound_name)
    
    ids = chempider_local_db.find_compound_ids_by_name_mass(compound_name, compound_mass)    
    if len(ids) == 0:
        print("Not found in local db, searching online")
        ids = chempider_web_db.find_compound_ids_by_name_mass(compound_name, compound_mass)
    
    csid_values.append(utils.CSID_list_to_string(ids))
    
compounds_table.insert(0, 'CSID', csid_values)
#display(compounds_table)
compounds_table.to_excel(export_file + "_step2.xlsx")
print("Done")

# Step 3: Find IUPAC Name

In [ ]:
#compounds_table = pd.read_excel(export_file + "_step2.xlsx")
iupac_values = []

num_rows = compounds_table.shape[0]
row_count = 0
for row_index, row_data in compounds_table.iterrows() :    
    row_count += 1
    compound_name = str(row_data.Name).lower()
    compound_csids = row_data['CSID']
    print('({:d}/{:d}) searching IUPAC Name '.format(row_count, num_rows) + compound_name)
    iupac_value = ""
    if not pd.isna(compound_csids):
        for csid in utils.parse_generated_CSID(compound_csids):
            compound = chempider_web_db.find_compound_by_id(csid)    
            iupac_value = iupac_value + compound.iupac_name + ";"
        
        
    iupac_values.append(iupac_value)
    
compounds_table.insert(0, 'IUPAC Name', iupac_values)
#display(compounds_table)
compounds_table.to_excel(export_file + "_step3.xlsx")
print("Done.")

# Step 4: Find External references from chemspi web search

In [ ]:
compounds_table = pd.read_excel(export_file + "_step3.xlsx")
external_databases = ['Human Metabolome Database', 'KEGG', 'LipidMAPS']
external_db_info = dict()
for db in external_databases:
    external_db_info[db] = []
        
num_rows = compounds_table.shape[0]
row_count = 0
for row_index, row_data in compounds_table.iterrows() :
    # search for external references from known databases
    row_count += 1
    print('({:d}/{:d}) Finding external references for '.format(row_count, num_rows) + row_data.Name)
    external_databases_result = {}
    for db in external_databases:
        external_databases_result[db] = []
    
    for csid in utils.parse_generated_CSID(row_data.CSID):
        print("CSID: ", csid)
        external_refs = chemsearch.http_find_compound_external_sources_ids(csid, external_databases)
        display(external_refs)
        
        #if len(warnings) > 0 :
        #    display(warnings)
        
        for db, values in external_refs.items():
            external_databases_result[db].extend(values)
    
    for db, values in external_databases_result.items():
        str_result = ""
        for v in values :
            str_result += str(v) + ";"
        external_db_info[db].append(str_result)
        
insert_pos = 1
for db_name, value in external_db_info.items():
    compounds_table.insert(insert_pos, db_name, value)
    insert_pos += 1

#display(compounds_table)
compounds_table.to_excel(export_file + "_step4.xlsx")
print("Done!")

# Step 5: Search not found HMDB metabolites

In [ ]:
compounds_table = pd.read_excel(export_file + "_step4.xlsx")
#display(compounds_table)
import hmdb_web_db as hmdb_web
num_rows = compounds_table.shape[0]
row_count = 0
for row_index, row_data in compounds_table.iterrows() :
    hmdb_info = row_data['Human Metabolome Database']
    print('({:d}/{:d})Missing HMDB id: '.format(row_count, num_rows), row_data.Name)
    row_count+=1
    if pd.isna(hmdb_info) or len(hmdb_info) == 0:
        if not row_data.Name.isnumeric():
            print("Searching by Name")
            metabolites = hmdb_web.search_metabolites_by_name_mass(row_data.Name, row_data['Molecular Weight'])
            compounds_table.loc[row_index, "Human Metabolome Database"] = utils.CSID_list_to_string(metabolites)
        else:
            print("Searching by IUPAC Name")
            iupac_names = row_data["IUPAC Name"]
            names_list = ""
            for iupac in iupac_names.split(';'):
                metabolites = hmdb_web.search_metabolites_by_name_mass(iupac, row_data['Molecular Weight'])
                if len(metabolites) > 0:     
                    names_list += utils.CSID_list_to_string(metabolites)
            compounds_table.loc[row_index, "Human Metabolome Database"] = names_list

compounds_table.to_excel(export_file + "_step5.xlsx")
print("Done!")

# Step 6: Search HMDB info

In [ ]:
import hmdb_web_db as hmdb_web

#compounds_table = pd.read_excel(export_file + "_step5.xlsx")

info_titles = ["Super Class", "Class", "Sub Class"]
compounds_columns = ["Name", "IUPAC Name", "Human Metabolome Database"]

columns = compounds_columns + info_titles
output_data = {title : [] for title in columns}

#####
num_rows = compounds_table.shape[0]
row_count = 0
for row_index, row_data in compounds_table.iterrows() :
    hmdb_ids = row_data['Human Metabolome Database']
    print('({:d}/{:d})Searching HMDB info: '.format(row_count, num_rows), hmdb_ids)
    row_count+=1
    
    #### search hmdb info
    hmdb_info = None
    if not pd.isna(hmdb_ids) and len(str(hmdb_ids)) > 0:
        for hmdb_id in hmdb_ids.split(';'):
            hmdb_info = hmdb_web.search_metabolite_info_by_id(hmdb_id, info_titles)
            print(hmdb_info)
            break
    
    #### set hmdb data
    if hmdb_info != None:        
        print(hmdb_info)
        for column_name, value in hmdb_info.items():        
            output_data[column_name].append(value)
    else:
        for column_name in info_titles:
            output_data[column_name].append("")
        
    #### set compounds data
    for column_name in compounds_columns:
        output_data[column_name].append(row_data[column_name])
        
            
result = pd.DataFrame.from_dict(output_data)
result.to_excel(export_file + "_step6.xlsx")
display(result)
   

#compounds_table.to_excel(export_file + "_step6.xlsx")
print("Done!")